In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")
import nltk
from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Reading Gene Dataset and EDA

In [ ]:
df_tab = pd.read_csv('/kaggle/input/msk-redefining-cancer-treatment/training_variants.zip')
df_tab.head()

In [ ]:
df_tab.shape

In [ ]:
# Count of Unique Values of Gene
df_tab["Gene"].value_counts()

In [ ]:
# Count of Unique Values of Variation
df_tab["Variation"].value_counts()

In [ ]:
# Count of Unique Values of Class
df_tab["Class"].value_counts()

In [ ]:
df_tab.isnull().sum()

## Reading Text Dataset and EDA

In [ ]:
df_text =pd.read_csv("/kaggle/input/msk-redefining-cancer-treatment/training_text.zip",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
df_text.head()

In [ ]:
df_text.shape

## Merge Datasets

In [ ]:
df_merge = pd.merge(df_tab,df_text, on="ID",how="left")
df_merge.head()

In [ ]:
# lowercase column names
df_merge.columns = df_merge.columns.str.lower()

## Clean Text Column

In [ ]:
df_merge.isnull().sum()

In [ ]:
df_merge[df_merge.isnull().any(axis=1)]

In [ ]:
# Fill NaN values with Gene & Variation 
df_merge.loc[df_merge["text"].isnull(),"text"] = df_merge["gene"] + " " + df_merge["variation"]

In [ ]:
df_merge.isnull().sum()

In [ ]:
# lowercase 
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(i.lower() for i in x.split()))
df_merge.head()

In [ ]:
# remove punctuations
df_merge["text"] = df_merge["text"].str.replace("[^\w\s]","")
df_merge.head()

In [ ]:
# remove numeric values
df_merge["text"] = df_merge["text"].str.replace("\d","")
df_merge.head()

In [ ]:
# remove stopwords: the,a,an etc.
nltk.download("stopwords")
from nltk.corpus import stopwords
sw = stopwords.words("english")

In [ ]:
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(i for i in x.split() if i not in sw))
df_merge.head()

## Tokenization
* Split sentence to words

In [ ]:
nltk.download("punkt")
import textblob
from textblob import TextBlob

In [ ]:
# an example:
TextBlob(df_merge["text"][1]).words

In [ ]:
df_merge["text"] = df_merge["text"].apply(lambda x: TextBlob(x).words)
df_merge.head()

## Lemmatization
* Create root of the word

In [ ]:
from textblob import Word
nltk.download("wordnet")

In [ ]:
df_merge["text"] = df_merge["text"].apply(lambda x: " ".join(Word(word).lemmatize() for word in x))
df_merge.head()

In [ ]:
df_merge.to_pickle("clean_text.pkl")

## Split Dataset 

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_merge[["gene","variation","text"]],
                                                                    df_merge["class"], 
                                                                    random_state = 7,
                                                                    stratify=df_merge["class"],
                                                                    test_size=0.2)

In [ ]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [ ]:
train_x.head()

## TF-IDF

In [ ]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x.text)

In [ ]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

In [ ]:
len(tf_idf_word_vectorizer.get_feature_names())

In [ ]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x.text)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x.text)

In [ ]:
loj = LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word, train_y)

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics

In [ ]:
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

In [ ]:
train_x.head()

In [ ]:
train_y.head()

In [ ]:
test_x.head()

In [ ]:
df_merge.head()

In [ ]:
df_merge.shape

In [ ]:
df_merge.dtypes

In [ ]:
gene_enc = pd.get_dummies(df_merge["gene"],drop_first=True)

In [ ]:
variation_enc = pd.get_dummies(df_merge["variation"],drop_first=True)

In [ ]:
df_enc = pd.concat([gene_enc,variation_enc],axis=1)

In [ ]:
df_enc2 = pd.concat([df_merge["class"],df_enc],axis=1)
df_enc2.head()

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_enc2.drop("class",axis=1),
                                                                    df_enc2["class"], 
                                                                    random_state = 7,
                                                                    stratify=df_merge["class"],
                                                                    test_size=0.2)

In [ ]:
loj = LogisticRegression()
loj_model = loj.fit(train_x, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           test_x, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

In [ ]:
tf_idf_word_vectorizer = TfidfVectorizer()
text_enc = tf_idf_word_vectorizer.fit_transform(df_merge.text)

In [ ]:
text_enc_df = pd.DataFrame(text_enc.toarray())

In [ ]:
df_enc3 = pd.concat([df_enc2,text_enc_df],axis=1)
df_enc3.head()

In [ ]:
# train_x, test_x, train_y, test_y = model_selection.train_test_split(df_enc3.drop("class",axis=1),
#                                                                     df_enc3["class"], 
#                                                                     random_state = 7,
#                                                                     stratify=df_merge["class"],
#                                                                     test_size=0.2)